In [44]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 19 16:55:30 2023

@author: maggie

Updated 5/14/2024 - Charlotte Stanley

"""

import arcpy
import pandas as pd
import os

In [3]:
"""
Changes made by Charlotte - Feb-Apr 2024

1. Clean up attributes of 8 separate treatment datasets - merge together
2. ITS treatments: remove duplicate polygons
3. Create two treatment datasets: one with and without Cal Fire data
    a) Run two versions of script
5. Correction: Fixed forest area to start removing high severity starting one year later (keep 1999 high severity in baseline forest area)
6. Correction: Fixed error when merging data from FACTS hazardous fuels dataset and timber harvest dataset. 
   This had been excluding all treatments in the hazardous fuels dataset in the final treatments shapefile, undercounting total treatments. 
7. Keep original treatment names in final shapefile
8. Remove points/lines buffer
9. Run with new ownership data (including private timberlands)
10. Run through 2022

(CHANGE NO LONGER NEEDED) ITS treatments: manually remove select polygons from Kristen Shive (mislabeled in database) - 
(CHANGE NO LONGER NEEDED) Remove all Cal Fire treatments in ITS

"""


'\nChanges made by Charlotte - Feb-Apr 2024\n\n1. Removed duplicate treatments (the same treatments that appeared in both FACTS and ITS)\n2. Fixed forest area to start removing high severity starting one year later (keep 1999 high severity in baseline forest area)\n3. Remove Cal Fire treatments completely\n4. Fixed error when merging data from FACTS hazardous fuels dataset and timber harvest dataset. \n   This had been excluding all treatments in the hazardous fuels dataset in the final treatments shapefile, undercounting total treatments. \n5. Keep original treatment names in final shapefile\n6. Need to manually remove select polygons from Kristen Shive (mislabeled in database)\n'

In [45]:
# Set Variables
arcpy.env.overwriteOutput = 1
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False # don't add outputs to map 


## load data

In [46]:
# Workspace
datagdb = r"N:\Sierra_Nevada\Projects\State of the Sierra\State of the Sierra\SOS_cs.gdb"
ws = datagdb
tables = datagdb
scratch = r"N:\Sierra_Nevada\Projects\State of the Sierra\State of the Sierra\scratch.gdb"

# Set Workspace
arcpy.env.workspace = ws

In [54]:
# Load Datasets

# Vegetation
Calveg = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\FINAL_CALVEG_1999_Current_mrg_YPMC_diss.shp'

# Treatments
trt_blm = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\BLM_Treatments_SoS_2011_2023.shp'
trt_blodgett = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\Blodgett_Treatments_SoS_FINAL.shp'
trt_cf_thp = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\CAL_FIRE_All_THPs_SoS_FINAL.shp'
trt_cf_rx = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\CAL_FIRE_Rx_SoS_FINAL.shp'
trt_facts_hf = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\FACTS_HazFuel_clean_SoS_FINAL.shp'
trt_facts_th = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\FACTS_TimberHarvest_cleanSoS_FINAL.shp'
trt_its = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\ITS_CNRA_Other_Treatments_SoS_FINAL.shp'
trt_nps = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\NPS_Treatments_2001_2022_SoS_FINAL.shp'
trt_list = [trt_blm, trt_blodgett, trt_cf_thp, trt_cf_rx, trt_facts_hf, trt_facts_th, trt_its, trt_nps]

In [86]:
# Severity
all_sev = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\FireSeverity_2000_2022_SN_mrg.shp'

In [105]:
# load cpad data
# includes: CA, NV, private timber, private other, UC Berkeley forests
cpad = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\Holdings_CA_NV_20240513.shp'

In [48]:
# Set coordinate system
arcpy.env.outputCoordinateSystem = Calveg

sr_ref = arcpy.Describe(Calveg).spatialReference

In [99]:
# Set years of interest
lst_years = list(range(2000, 2023))

## Treatment datasets clean up

In [50]:
# Dictionary to map field names

field_mapping_blm = {'TypeName':'Name', 'CategoryNa':'Category'}
field_mapping_blodgett = {'treatment':'Name'}
field_mapping_cf_thp = {'SILVI_1':'Name', 'SILVI_CAT':'Category'}
field_mapping_cf_rx = {}
field_mapping_facts_hf = {'ACTIVITY':'Name', 'TREATMENT_':'Category', 'ACTIVITY_C':'Code'}
field_mapping_facts_th = {'ACTIVITY_N':'Name', 'TREATMENT_':'Category', 'ACTIVITY_2':'Code'}
field_mapping_its = {'ACTIVITY_D':'Name', 'ACTIVITY_C':'Category'}
field_mapping_nps = {'TreatmentT':'Name', 'TreatmentC':'Category'}

In [55]:
trt_list = trt_list[-2:-1]
trt_list

['N:\\Sierra_Nevada\\Projects\\State of the Sierra\\Inputs_Final_20240510\\ITS_CNRA_Other_Treatments_SoS_FINAL.shp']

In [56]:
# for all treatment shapefiles:
# change field names, delete extra fields, calculate acres

for trt in trt_list:
    # get name of file
    trt_suffix = trt.split('\\')[-1][:-4]
    print(trt_suffix)
    
    # describe spatial reference
    sr = arcpy.Describe(trt).spatialReference
    
    # if coordinate system is correct
    if sr.type == sr_ref.type:
        print('Coordinate system correct')
        # copy features
        trt_copy = ws + '\\' + trt_suffix
        arcpy.management.CopyFeatures(trt, trt_copy)
    
    # if coordinate system is not correct
    else: 
        print('Reproject coordinate system')
        # reproject features
        trt_copy = ws + '\\' + trt_suffix
        arcpy.Project_management(trt, trt_copy, sr_ref)
    
    # for all new fcs
    
    # set field_mapping dictionary (to map new field names)
    if "blm" in trt_copy.lower():
        field_mapping = field_mapping_blm
    elif "blodgett" in trt_copy.lower():
        field_mapping = field_mapping_blodgett
    elif "cal_fire_all_thp" in trt_copy.lower():
        field_mapping = field_mapping_cf_thp
    elif "cal_fire_rx" in trt_copy.lower():
        field_mapping = field_mapping_cf_rx
    elif "facts_haz" in trt_copy.lower():
        field_mapping = field_mapping_facts_hf
    elif "facts_timber" in trt_copy.lower():
        field_mapping = field_mapping_facts_th
    elif "its" in trt_copy.lower():
        field_mapping = field_mapping_its
    elif "nps" in trt_copy.lower():
        field_mapping = field_mapping_nps
    else:
        print('Continue')
    
    # delete these field names if they already exist
    arcpy.management.DeleteField(trt_copy, ['Name','Category','Code'], 'DELETE_FIELDS')
        
    # rename fields (using field_mapping dictionary)
    print('Rename fields')

    for existing_field, new_field in field_mapping.items():
        try:
            arcpy.management.AlterField(trt_copy, existing_field, new_field, new_field)
            print(f"Field '{existing_field}' altered to '{new_field}'")
        except arcpy.ExecuteError:
            print(f"Field '{existing_field}' does not exist in the feature class.")

    # add fields
    if not arcpy.ListFields(trt_copy, 'Name'):
        arcpy.management.AddField(trt_copy, 'Name', 'TEXT')
        
    if not arcpy.ListFields(trt_copy, 'Category'):
        arcpy.management.AddField(trt_copy, 'Category', 'TEXT')
    
    if not arcpy.ListFields(trt_copy, 'Code'):
        arcpy.management.AddField(trt_copy, 'Code', 'TEXT')
        
    # keep relevant fields
    print('Delete fields')
    if "its" in trt_copy.lower():
        arcpy.management.DeleteField(trt_copy, [ 'Name', 'Category', 'Code', 'Year', 'UCB_TREATM', 'LATITUDE', 'LONGITUDE'], 'KEEP_FIELDS')
    else:
        arcpy.management.DeleteField(trt_copy, [ 'Name', 'Category', 'Code', 'Year', 'UCB_TREATM'], 'KEEP_FIELDS')

    # calculate acres
    print('Calculate acres')
    arcpy.management.CalculateGeometryAttributes(trt_copy, "US_SURV_AC AREA", "", "ACRES_US", sr_ref)
    print('\n')

ITS_CNRA_Other_Treatments_SoS_FINAL
Coordinate system correct
Rename fields
Field 'ACTIVITY_D' altered to 'Name'
Field 'ACTIVITY_C' altered to 'Category'
Delete fields
Calculate acres




In [58]:
# load new clean datasets
trt_blm = ws + '\\BLM_Treatments_SoS_2011_2023'
trt_blodgett = ws + '\\Blodgett_Treatments_SoS_FINAL'
trt_cf_thp = ws + '\\CAL_FIRE_All_THPs_SoS_FINAL'
trt_cf_rx = ws + '\\CAL_FIRE_Rx_SoS_FINAL'
trt_facts_hf = ws + '\\FACTS_HazFuel_clean_SoS_FINAL'
trt_facts_th = ws + '\\FACTS_TimberHarvest_cleanSoS_FINAL'
trt_its = ws + '\\ITS_CNRA_Other_Treatments_SoS_FINAL'
trt_nps = ws + '\\NPS_Treatments_2001_2022_SoS_FINAL'
trt_list = [trt_blm, trt_blodgett, trt_cf_thp, trt_cf_rx, trt_facts_hf, trt_facts_th, trt_its, trt_nps]

In [35]:
# for cal fire rx only
# fill in treatment type with "Prescribed burn"
arcpy.management.CalculateField(trt_cf_rx, 'Name', '"Prescribed Burn"', 'PYTHON3', "", 'TEXT')


<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\CAL_FIRE_Rx_SoS_FINAL'>

## ITS treatments - remove duplicates

In [59]:
# copy feature class
trt_its_copy = ws + '\\ITS_CNRA_Other_Treatments_SoS_copy'
arcpy.management.CopyFeatures(trt_its, trt_its_copy)

<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ITS_CNRA_Other_Treatments_SoS_copy'>

In [60]:
# load input
inFeatures = trt_its_copy

# add fields - calculate area
arcpy.management.CalculateGeometryAttributes(inFeatures, "GIS_ACRES AREA", "", "ACRES")

# round acres, lat, lon to 4 decimals
arcpy.management.CalculateField(inFeatures, 'LAT_RND4', "round(!LATITUDE!,4)", "PYTHON3")
arcpy.management.CalculateField(inFeatures, 'LON_RND4', "round(!LONGITUDE!,4)", "PYTHON3")
arcpy.management.CalculateField(inFeatures, 'AC_RND4', "round(!US_SURV_AC!,4)", "PYTHON3")


<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ITS_CNRA_Other_Treatments_SoS_copy'>

In [72]:
# find identical polygons (identical based on fields: treatment type, year, rounded acres, rounded lat, rounded lon)
fields = ['Name', 'Category', 'UCB_TREATM', 'Year', 'AC_RND4','LAT_RND4','LON_RND4']
out_dataset = ws + '\\ITS_CNRA_Other_Treatments_SoS_findIdentical'
arcpy.management.FindIdentical(trt_its_copy, out_dataset, fields, "", "", "ONLY_DUPLICATES")

<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ITS_CNRA_Other_Treatments_SoS_findIdentical'>

In [76]:
# get list of OBJECTIDs of identical polygons to review outputs 
out_dataset = ws + '\\ITS_CNRA_Other_Treatments_SoS_sp_findIdentical'
fieldname = r'IN_FID'

# Create an empty list
value_list = []

# Iterate through the field values and append to the list
with arcpy.da.SearchCursor(out_dataset, [fieldname]) as cursor:
    for row in cursor:
        value_list.append(row[0])

print(value_list)

[97, 188, 98, 189, 99, 190, 100, 191, 101, 192, 102, 193, 103, 194, 104, 195, 105, 196, 106, 197, 107, 198, 108, 199, 2603, 2665, 2605, 2668, 2610, 2672, 2611, 2673, 2612, 2674, 2613, 2675]


In [ ]:
# review outputs 
# identical polygons = OBJECTID: 21 / 37 

In [77]:
# delete select polygons - user defined list

# Create a list of OBJECTIDs to delete
objectid_list = [37]

# Delete polygons with matching OBJECTIDs
with arcpy.da.UpdateCursor(trt_its_copy, 'OBJECTID') as cursor:
    for row in cursor:
        if row[0] in objectid_list:
            cursor.deleteRow()

In [78]:
# split apart multipart to singlepart polygons
# to remove same treatments that are sometimes combined into one multi-part polygon
outFeatures = ws + '\\ITS_CNRA_Other_Treatments_SoS_sp'
arcpy.management.MultipartToSinglepart(trt_its_copy, outFeatures)

<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ITS_CNRA_Other_Treatments_SoS_sp'>

In [79]:
# calculate geometries for single part polygons (area, lat, lon)
arcpy.management.CalculateGeometryAttributes(outFeatures, "GIS_ACRES_2 AREA", "", "ACRES", "", "SAME_AS_INPUT")
arcpy.management.CalculateGeometryAttributes(outFeatures, "GIS_LAT CENTROID_Y", "", "", "", "DD")
arcpy.management.CalculateGeometryAttributes(outFeatures, "GIS_LON CENTROID_X", "", "", "", "DD")


<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ITS_CNRA_Other_Treatments_SoS_sp'>

In [80]:
# find identical polygons in singlepart dataset (identical based on fields: treatment type, year, rounded acres, rounded lat, rounded lon)
fields = ['Name', 'Category', 'UCB_TREATM', 'Year', 'GIS_ACRES_2','GIS_LAT','GIS_LON']
in_dataset = ws + '\\ITS_CNRA_Other_Treatments_SoS_sp'
out_dataset = ws + '\\ITS_CNRA_Other_Treatments_SoS_sp_findIdentical'
arcpy.management.FindIdentical(in_dataset, out_dataset, fields, "", "", "ONLY_DUPLICATES")

# review outputs - delete all

<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ITS_CNRA_Other_Treatments_SoS_sp_findIdentical'>

In [81]:
# delete identical polygons
fields = ['Name','Category','UCB_TREATM','Year','GIS_ACRES_2','GIS_LAT','GIS_LON']
outFeatures = ws + '\\ITS_CNRA_Other_Treatments_SoS_sp'
arcpy.management.DeleteIdentical(outFeatures, fields)


<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ITS_CNRA_Other_Treatments_SoS_sp'>

In [83]:
# delete fields
in_features = ws + '\\ITS_CNRA_Other_Treatments_SoS_sp'
arcpy.management.DeleteField(in_features, [ 'Name', 'Category', 'Code', 'Year', 'UCB_TREATM'], 'KEEP_FIELDS')


<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ITS_CNRA_Other_Treatments_SoS_sp'>

In [84]:
# load new clean datasets
trt_its = ws + '\\ITS_CNRA_Other_Treatments_SoS_sp'
trt_list = [trt_blm, trt_blodgett, trt_cf_thp, trt_cf_rx, trt_facts_hf, trt_facts_th, trt_its, trt_nps]

## combine treatment data

### v1 - without calfire

In [95]:
# create treatment list without calfire datasets
trt_list_noCF = [item for item in trt_list if "CAL_FIRE" not in item]

In [97]:
# v1 - no calfire data
ALL_TREATMENTS_noCF = os.path.join(datagdb, 'ALL_TREATMENTS_noCF')

# merge
arcpy.management.Merge(trt_list_noCF, ALL_TREATMENTS_noCF, "" , 'ADD_SOURCE_INFO')

<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ALL_TREATMENTS_noCF'>

### v2 - with calfire

In [98]:
# v1 - no calfire data
ALL_TREATMENTS_CF = os.path.join(datagdb, 'ALL_TREATMENTS_CF')

# merge
arcpy.management.Merge(trt_list, ALL_TREATMENTS_CF, "" , 'ADD_SOURCE_INFO')

<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\State of the Sierra\\SOS_cs.gdb\\ALL_TREATMENTS_CF'>

## annual forest area

In [89]:
# annual forest area - exclude high severity fire from previous year 

def create_forest_area_by_year_wHiGH():
    '''
    This function creates a feature class for each year that EXCLUDES high severity area from 
    previous years but includes high severity from CURRENT year. Uses the merged fire severity layer. 
    '''

    for yr in range(2000, 2023):
        sel = arcpy.SelectLayerByAttribute_management(all_sev, 'NEW_SELECTION', "FIRE_YEAR>'"+str(1999)+"' And FIRE_YEAR<'"+str(yr)+"' And BURNSEV=4")
        print(f'erasing high severity for years greater than 1999 and less than {yr}')
        arcpy.Erase_analysis(Calveg, sel, f'processing_forest_{yr}')
        

In [90]:
create_forest_area_by_year_wHiGH()

erasing high severity for years greater than 1999 and less than 2000
erasing high severity for years greater than 1999 and less than 2001
erasing high severity for years greater than 1999 and less than 2002
erasing high severity for years greater than 1999 and less than 2003
erasing high severity for years greater than 1999 and less than 2004
erasing high severity for years greater than 1999 and less than 2005
erasing high severity for years greater than 1999 and less than 2006
erasing high severity for years greater than 1999 and less than 2007
erasing high severity for years greater than 1999 and less than 2008
erasing high severity for years greater than 1999 and less than 2009
erasing high severity for years greater than 1999 and less than 2010
erasing high severity for years greater than 1999 and less than 2011
erasing high severity for years greater than 1999 and less than 2012
erasing high severity for years greater than 1999 and less than 2013
erasing high severity for years gr

## separate severity by year

In [106]:
# function to separate treatments by year 
def separate_severity():
    '''This function creates separate severity feature classes for each year. 
    The new feature classes are saved in the processing.gdb. 
    '''
    
    for i in lst_years:
        # select by year
        sel = arcpy.management.SelectLayerByAttribute(all_sev, 'NEW_SELECTION', "FIRE_YEAR= '"+str(i)+"'")
        arcpy.management.CopyFeatures(sel, os.path.join(datagdb, "SEVERITY_" + str(i)))
        print (f'finished saving SEVERITY_{i}')

In [108]:
# run function to separate treatments
separate_severity()

finished saving SEVERITY_2000
finished saving SEVERITY_2001
finished saving SEVERITY_2002
finished saving SEVERITY_2003
finished saving SEVERITY_2004
finished saving SEVERITY_2005
finished saving SEVERITY_2006
finished saving SEVERITY_2007
finished saving SEVERITY_2008
finished saving SEVERITY_2009
finished saving SEVERITY_2010
finished saving SEVERITY_2011
finished saving SEVERITY_2012
finished saving SEVERITY_2013
finished saving SEVERITY_2014
finished saving SEVERITY_2015
finished saving SEVERITY_2016
finished saving SEVERITY_2017
finished saving SEVERITY_2018
finished saving SEVERITY_2019
finished saving SEVERITY_2020
finished saving SEVERITY_2021
finished saving SEVERITY_2022


## dissolve cpad

In [111]:
cpad_dis = r'N:\Sierra_Nevada\Projects\State of the Sierra\Inputs_Final_20240510\Holdings_CA_NV_dissolve_20240513.shp'
arcpy.management.Dissolve(cpad, cpad_dis, 'FTE_Type')

<Result 'N:\\Sierra_Nevada\\Projects\\State of the Sierra\\Inputs_Final_20240510\\Holdings_CA_NV_dissolve_20240513.shp'>

## intersect forest-treaments-severity-cpad for final results

### v1 - without cal fire treatments

In [102]:
# function to separate treatments by year 
def separate_treatment():
    '''This function creates separate treatment feature classes for each year. 
    The new feature classes are saved in the processing.gdb. 
    '''
    
    for yr in lst_years:
        # select by year
        sel = arcpy.management.SelectLayerByAttribute(ALL_TREATMENTS_noCF, 'NEW_SELECTION', f" Year = {yr} ")
        arcpy.management.CopyFeatures(sel, f'tr{yr}')
        arcpy.management.RepairGeometry(f'tr{yr}', 'DELETE_NULL', "ESRI")
        
        # dissolve by crosswalked treatment type
        arcpy.management.Dissolve(f'tr{yr}', os.path.join(ws, f'treatment{yr}_noCF'), 'UCB_TREATM', '', 'MULTI_PART', 'DISSOLVE_LINES')
        print(f'finished saving treatment{yr}')
 

In [103]:
# run function to separate treatments
separate_treatment()

finished saving treatment2000
finished saving treatment2001
finished saving treatment2002
finished saving treatment2003
finished saving treatment2004
finished saving treatment2005
finished saving treatment2006
finished saving treatment2007
finished saving treatment2008
finished saving treatment2009
finished saving treatment2010
finished saving treatment2011
finished saving treatment2012
finished saving treatment2013
finished saving treatment2014
finished saving treatment2015
finished saving treatment2016
finished saving treatment2017
finished saving treatment2018
finished saving treatment2019
finished saving treatment2020
finished saving treatment2021
finished saving treatment2022


In [109]:
# function to intersect (1) annual forest (with HS removed, called "processing_forest_yr") + severity + cpad + treatment 

def assign_severity_treatment_fte_by_year():  
    '''
    This function performs 3 consecutive identity analyses that result in an output
    of the spatial intersections of the cpad, severity, and treatment layers
    '''
    #do extract by mask on each year, write output fc to memory
    arcpy.env.workspace = ws
    
    for i in lst_years:
        print ('severity identity for' + str(i))
        sevs = os.path.join(datagdb, "SEVERITY_" + str(i) )
        outpts = os.path.join(scratch, "assigned_severity_noCF_" + str(i))
        arcpy.analysis.Identity(f'processing_forest_{i}', sevs, outpts)
        print(f'then assigning cpad to {i}')
        arcpy.analysis.Identity(outpts, cpad_dis, os.path.join(scratch, f"assigned_severity_fte_noCF_{i}"))
        
        print(f'finally assigned treatment to {i}')
        arcpy.analysis.Identity(os.path.join(scratch, f"assigned_severity_fte_noCF_{i}"), f'treatment{i}_noCF', f'values_noCF_{i}')
        
        
        

In [110]:
# run function 
print('These functions assigns attributes based on spatial intersections using the queried forest raster with high severity for each year')
assign_severity_treatment_fte_by_year()

These functions assigns attributes based on spatial intersections using the queried forest raster with high severity for each year
severity identity for2000
then assigning cpad to 2000
finally assigned treatment to 2000
severity identity for2001
then assigning cpad to 2001
finally assigned treatment to 2001
severity identity for2002
then assigning cpad to 2002
finally assigned treatment to 2002
severity identity for2003
then assigning cpad to 2003
finally assigned treatment to 2003
severity identity for2004
then assigning cpad to 2004
finally assigned treatment to 2004
severity identity for2005
then assigning cpad to 2005
finally assigned treatment to 2005
severity identity for2006
then assigning cpad to 2006
finally assigned treatment to 2006
severity identity for2007
then assigning cpad to 2007
finally assigned treatment to 2007
severity identity for2008
then assigning cpad to 2008
finally assigned treatment to 2008
severity identity for2009
then assigning cpad to 2009
finally assign

In [122]:
# combine all results into one table

tables_folder = r'N:\Sierra_Nevada\Projects\State of the Sierra\State of the Sierra\TNC_tables_polygon\noCF'

def create_df_by_year():
    '''
    This function creates a dataframe from the output of the spatial intersections of 
    the cpad, severity, and treatment layers, then cleans the dataframe so that it has no blank cells. 
    It also replaces values codes with text descriptions for severity. 
    '''
    
    lst_dfs = []
    for i in lst_years:
        print ('reading data from fc for ' + str(i))
        
        # change input name for CF / yes CF 
        df = pd.DataFrame([[j[0], j[1], j[2], j[3]] for j in arcpy.da.SearchCursor(f'values_noCF_{i}', ['FTE_Type', 'SevClass','UCB_TREATM', 'Shape_Area'])])  #shape area, ...not point id..
        
        df.columns = ['FTE','SEVERITY','Treatment', 'Shape_Area']
        df['Acres'] = df['Shape_Area']*0.0002471054
        df = df[['FTE','SEVERITY', 'Treatment', 'Acres']]
        
        #filling empty cells
        print('replacing blank cells with No Treatment')
        df['Treatment'] = df['Treatment'].replace("", 'No Treatment')
        print('Removing Nans and replaceing with No Treatment')
        df['Treatment'] = df['Treatment'].fillna('No Treatment')
        print('replacing blank cells with No FTE LookUp')
        df['FTE'] = df['FTE'].replace("", 'No FTE LookUp')
        print('Removing Nans and replaceing with No FTE LookUp')
        df['FTE']= df['FTE'].fillna('No FTE LookUp')
        df['SEVERITY']= df['SEVERITY'].replace("", 'Unburned')
        
        df.to_csv(os.path.join(tables_folder, f'values_noCF_{i}.csv'), index = False)
        
        lst_dfs.append(df)
    
    return lst_dfs

In [123]:
# run function to combine 
lst_dfs = create_df_by_year()

reading data from fc for 2000
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2001
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2002
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2003
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2004
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and 

### v2 - with cal fire treatments

In [116]:
def separate_treatment_CF():
    '''This function creates separate treatment feature classes for each year. 
    The new feature classes are saved in the processing.gdb. 
    '''
    
    for yr in lst_years:
        # select by year
        sel = arcpy.management.SelectLayerByAttribute(ALL_TREATMENTS_CF, 'NEW_SELECTION', f" Year = {yr} ")
        arcpy.management.CopyFeatures(sel, f'tr_CF_{yr}')
        arcpy.management.RepairGeometry(f'tr_CF_{yr}', 'DELETE_NULL', "ESRI")
        
        # dissolve by crosswalked treatment type
        arcpy.management.Dissolve(f'tr_CF_{yr}', os.path.join(ws, f'treatment{yr}_CF'), 'UCB_TREATM', '', 'MULTI_PART', 'DISSOLVE_LINES')
        print(f'finished saving treatment{yr}')
 


In [117]:
separate_treatment_CF()

finished saving treatment2000
finished saving treatment2001
finished saving treatment2002
finished saving treatment2003
finished saving treatment2004
finished saving treatment2005
finished saving treatment2006
finished saving treatment2007
finished saving treatment2008
finished saving treatment2009
finished saving treatment2010
finished saving treatment2011
finished saving treatment2012
finished saving treatment2013
finished saving treatment2014
finished saving treatment2015
finished saving treatment2016
finished saving treatment2017
finished saving treatment2018
finished saving treatment2019
finished saving treatment2020
finished saving treatment2021
finished saving treatment2022


In [118]:
# function to intersect (1) annual forest (with HS removed, called "processing_forest_yr") + severity + cpad + treatment 

def assign_severity_treatment_fte_by_year_CF():  
    '''
    This function performs 3 consecutive identity analyses that result in an output
    of the spatial intersections of the cpad, severity, and treatment layers
    '''
    #do extract by mask on each year, write output fc to memory
    arcpy.env.workspace = ws
    
    for i in lst_years:
        print ('severity identity for' + str(i))
        sevs = os.path.join(datagdb, "SEVERITY_" + str(i) )
        outpts = os.path.join(scratch, "assigned_severity_CF_" + str(i))
        arcpy.analysis.Identity(f'processing_forest_{i}', sevs, outpts)
        print(f'then assigning cpad to {i}')
        arcpy.analysis.Identity(outpts, cpad_dis, os.path.join(scratch, f"assigned_severity_fte_CF_{i}"))
        
        print(f'finally assigned treatment to {i}')
        arcpy.analysis.Identity(os.path.join(scratch, f"assigned_severity_fte_CF_{i}"), f'treatment{i}_CF', f'values_CF_{i}')
        
        
        

In [119]:
print('These functions assigns attributes based on spatial intersections using the queried forest raster with high severity for each year')
assign_severity_treatment_fte_by_year_CF()

These functions assigns attributes based on spatial intersections using the queried forest raster with high severity for each year
severity identity for2000
then assigning cpad to 2000
finally assigned treatment to 2000
severity identity for2001
then assigning cpad to 2001
finally assigned treatment to 2001
severity identity for2002
then assigning cpad to 2002
finally assigned treatment to 2002
severity identity for2003
then assigning cpad to 2003
finally assigned treatment to 2003
severity identity for2004
then assigning cpad to 2004
finally assigned treatment to 2004
severity identity for2005
then assigning cpad to 2005
finally assigned treatment to 2005
severity identity for2006
then assigning cpad to 2006
finally assigned treatment to 2006
severity identity for2007
then assigning cpad to 2007
finally assigned treatment to 2007
severity identity for2008
then assigning cpad to 2008
finally assigned treatment to 2008
severity identity for2009
then assigning cpad to 2009
finally assign

In [124]:
# combine all results into one table

tables_folder = r'N:\Sierra_Nevada\Projects\State of the Sierra\State of the Sierra\TNC_tables_polygon\CF'

def create_df_by_year():
    '''
    This function creates a dataframe from the output of the spatial intersections of 
    the cpad, severity, and treatment layers, then cleans the dataframe so that it has no blank cells. 
    It also replaces values codes with text descriptions for severity. 
    '''
    
    lst_dfs = []
    for i in lst_years:
        print ('reading data from fc for ' + str(i))
        
        # change input name for CF / yes CF 
        df = pd.DataFrame([[j[0], j[1], j[2], j[3]] for j in arcpy.da.SearchCursor(f'values_CF_{i}', ['FTE_Type', 'SevClass','UCB_TREATM', 'Shape_Area'])])  #shape area, ...not point id..
        
        df.columns = ['FTE','SEVERITY','Treatment', 'Shape_Area']
        df['Acres'] = df['Shape_Area']*0.0002471054
        df = df[['FTE','SEVERITY', 'Treatment', 'Acres']]
        
        #filling empty cells
        print('replacing blank cells with No Treatment')
        df['Treatment'] = df['Treatment'].replace("", 'No Treatment')
        print('Removing Nans and replaceing with No Treatment')
        df['Treatment'] = df['Treatment'].fillna('No Treatment')
        print('replacing blank cells with No FTE LookUp')
        df['FTE'] = df['FTE'].replace("", 'No FTE LookUp')
        print('Removing Nans and replaceing with No FTE LookUp')
        df['FTE']= df['FTE'].fillna('No FTE LookUp')
        df['SEVERITY']= df['SEVERITY'].replace("", 'Unburned')
        
        df.to_csv(os.path.join(tables_folder, f'values_CF_{i}.csv'), index = False)
        
        lst_dfs.append(df)
    
    return lst_dfs

In [125]:
# run function to combine 
lst_dfs = create_df_by_year()

reading data from fc for 2000
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2001
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2002
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2003
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2004
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and 

reading data from fc for 2000
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2001
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2002
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2003
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and replaceing with No FTE LookUp
reading data from fc for 2004
replacing blank cells with No Treatment
Removing Nans and replaceing with No Treatment
replacing blank cells with No FTE LookUp
Removing Nans and 